In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as scipy_signal
from scipy.interpolate import PchipInterpolator 
import neurokit2 as nk

In [ ]:
fs_acc = 32  

for participant_id in participants:

    filtered_folder = os.path.join(DATA_FINAL_DIR, participant_id, "Filtered_Signals")
    os.makedirs(filtered_folder, exist_ok=True)

    for side in ["LEFT", "RIGHT"]:
        acc_path = os.path.join(
            DATA_FINAL_DIR,
            participant_id,
            "Signals",
            f"{participant_id}_{side}_acc.csv",
        )

        df_acc = pd.read_csv(acc_path)
        acc_x_raw = df_acc["acc_x"].values.copy()
        acc_y_raw = df_acc["acc_y"].values.copy()
        acc_z_raw = df_acc["acc_z"].values.copy()

        print(f"  {side}: Loaded {len(acc_x_raw):,} samples")

        lowcut = 0.1
        highcut = 10.0
        order = 2
        nyquist = fs_acc / 2

        b, a = scipy_signal.butter(
            order, [lowcut / nyquist, highcut / nyquist], btype="band"
        )

        acc_x_filtered = scipy_signal.filtfilt(b, a, acc_x_raw)
        acc_y_filtered = scipy_signal.filtfilt(b, a, acc_y_raw)
        acc_z_filtered = scipy_signal.filtfilt(b, a, acc_z_raw)

        acc_magnitude_raw = np.sqrt(acc_x_raw**2 + acc_y_raw**2 + acc_z_raw**2)

        acc_magnitude_filtered = np.sqrt(
            acc_x_filtered**2 + acc_y_filtered**2 + acc_z_filtered**2
        )

        # enmo
        enmo = np.clip(acc_magnitude_raw - 1.0, 0, None)
        artifact_mask = np.zeros(len(acc_x_raw), dtype=bool)
        artifact_mask |= acc_magnitude_raw > 8.0

        artifact_pct = artifact_mask.sum() / len(acc_x_raw) * 100
        if artifact_pct > 0:
            print(f"       Artifacts: {artifact_mask.sum():,} ({artifact_pct:.1f}%)")

        
        df_out = df_acc.copy()

        df_out["acc_x_raw"] = acc_x_raw
        df_out["acc_y_raw"] = acc_y_raw
        df_out["acc_z_raw"] = acc_z_raw
        df_out["acc_x_filtered"] = acc_x_filtered
        df_out["acc_y_filtered"] = acc_y_filtered
        df_out["acc_z_filtered"] = acc_z_filtered
        df_out["acc_magnitude"] = acc_magnitude_raw
        df_out["acc_magnitude_filtered"] = acc_magnitude_filtered
        df_out["enmo"] = enmo
        df_out["artifact"] = artifact_mask.astype(int)
        df_out = df_out.drop(columns=["acc_x", "acc_y", "acc_z"], errors="ignore")

        cols = [
            "participant_id",
            "side",
            "phase_start",
            "phase_end",
            "timestamp",
            "acc_x_raw",
            "acc_y_raw",
            "acc_z_raw",
            "acc_x_filtered",
            "acc_y_filtered",
            "acc_z_filtered",
            "acc_magnitude",
            "acc_magnitude_filtered",
            "enmo",
            "artifact",
            "phase",
        ]
        cols = [c for c in cols if c in df_out.columns]
        df_out = df_out[cols]